## Categorical Feature Encoding Project - 머신러닝 편 (성능개선 ver.1)
- 앞의 베이스라인 모델을 만들며 범주형 변수의 인코딩(Encoding)을 다루었다.
- 이번 노트북에서는 성능 향상을 위해 이전의 베이스라인 모델에 몇 가지 경우를 더 추가해보았다.
    - 1) 명목형 변수(`nom_*` 의 변수 개수 조정)
    - 2) 로지스틱 회귀의 하이퍼파라미터 튜닝

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

display(train.head())
display(test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [3]:
# 인코딩은 train과 test에 모두 해줄 것이므로 먼저 두개를 이어붙인다. target은 따로 빼서 저장
all_data = pd.concat([train, test], ignore_index = True)
all_data.drop(columns = 'target', axis = 1, inplace = True)
all_data.head(3)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2


In [4]:
print(all_data.shape)

(500000, 23)


In [5]:
# target
train_target = train['target']

## 1st. `nom_*` 변수의 개수에 따라 test score 비교
- 베이스라인 모델에서 했던 그대로, `nom_*` 변수의 개수를 조정하였다.
- 이 부분 외의 다른 코드는 베이스라인 모델에서 사용한 것과 똑같다. 모델도 기본 로지스틱 회귀 모형을 사용. -> **중복되는 코드이므로 함수화!**

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

In [7]:
### 범주형 데이터 전처리 함수
def cat_preprocessing(df, n_nom):
    # df: 전처리할 데이터
    # n_nom: 원하는 nom_* 변수의 개수 (만일 10을 넣으면 nom_0 ~ nom_9까지 포함하므로 베이스라인 모델과 동일함)
    # 반환되는 값은 train_sprs, test_sprs
    
    print('nom_* 변수는 nom_0 부터 nom_{0} 까지 사용되었습니다'.format(n_nom-1))
    print(' ')
    
    # bin_* 변수 전처리
    bin_cols = [f'bin_{i}' for i in range(0, 5)]
    bin_data = df[bin_cols]
    bin_data['bin_3'] = bin_data['bin_3'].apply(lambda x: 1 if x == 'T' else 0)
    bin_data['bin_4'] = bin_data['bin_4'].apply(lambda x: 1 if x == 'Y' else 0)
    
    # ord_* 변수 전처리
    ord_cols = [f'ord_{i}' for i in range(0, 6)] 
    ord_data = df[ord_cols]
    ord1_enc = {'Novice': 0, 'Contributor': 1, 'Expert': 2, 'Master': 3, 'Grandmaster': 4}
    ord2_enc = {'Freezing': 0 , 'Cold': 1, 'Warm': 2,  'Hot': 3, 'Boiling Hot': 4, 'Lava Hot': 5}
    ord_data.loc[:, 'ord_1'] = ord_data.loc[:, 'ord_1'].map(ord1_enc)
    ord_data.loc[:, 'ord_2'] = ord_data.loc[:, 'ord_2'].map(ord2_enc)
        ## 숫자 순위 부여
    encoder = LabelEncoder()
    ord_data['ord_3'] = encoder.fit_transform(ord_data['ord_3'].values)
    ord_data['ord_4'] = encoder.fit_transform(ord_data['ord_4'].values)
    ord_data['ord_5'] = encoder.fit_transform(ord_data['ord_5'].values)
        ## 숫자 스케일링
    scaler = StandardScaler()
    ord_data_scaled = scaler.fit_transform(ord_data)
    
    # 앞에서 만든 bin_data & ord_data 합침
    df2 = pd.concat([bin_data, pd.DataFrame(ord_data_scaled, columns = ord_data.columns)], axis = 1)
    
    #--------------- nom_* 변수에서 어떤 변수를 포함할지 선택------------------
    nom_cols = [f'nom_{i}' for i in range(0, n_nom)] # 변수 개수 조정 
    nm_list = ['day', 'month']
    nom_cols.extend(nm_list)
    nom_data = df[nom_cols]
   
    encoder = OneHotEncoder()
    nom_data_enc = encoder.fit_transform(nom_data)
    
    # df2와 nom_data_enc를 결합
    all_sprs = sparse.hstack([sparse.csr_matrix(df2), nom_data_enc], format = 'csr')
    
    # all_sprs는 앞 부분은 train / 뒷 부분은 test 데이터이므로 분할
    train_sprs = all_sprs[:len(train), :]
    test_sprs = all_sprs[len(train):, :]
    print('train data의 shape: ', train_sprs.shape)
    print('test data의 shape: ', test_sprs.shape)
    
    return train_sprs, test_sprs

In [8]:
### train, val set을 나눈 후에 기본 로지스틱 회귀 모델로 모델링 후, val set의 AUC score 출력 함수
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
    
def logistic_train(train_sprs, y, tsize = 0.2, n_iter = 100):

    # train, val set split
    X_train, X_val, y_train, y_val = train_test_split(train_sprs, y, stratify = y, random_state = 99, test_size = tsize)
    print('X_train의 shape: ', X_train.shape)
    print('X_val의 shape: ', X_val.shape)
    
    # 기본 모델링
    model = LogisticRegression(random_state = 99, max_iter = n_iter)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    
    # 성능 (AUC) 출력
    print(' ')
    print("val set's AUC score : {:.4f}".format(roc_auc_score(y_val, y_pred)))

### nom_* 변수에 따라 값을 뽑아보자! 

In [9]:
# baseline model과 동일
train_sprs, test_sprs = cat_preprocessing(all_data, 10)
logistic_train(train_sprs, train_target)

nom_* 변수는 nom_0 부터 nom_9 까지 사용되었습니다
 
train data의 shape:  (300000, 16306)
test data의 shape:  (200000, 16306)
X_train의 shape:  (240000, 16306)
X_val의 shape:  (60000, 16306)
 
val set's AUC score : 0.8004


In [10]:
# nom_9 제외
train_sprs, test_sprs = cat_preprocessing(all_data, 9)
logistic_train(train_sprs, train_target)

nom_* 변수는 nom_0 부터 nom_8 까지 사용되었습니다
 
train data의 shape:  (300000, 4238)
test data의 shape:  (200000, 4238)
X_train의 shape:  (240000, 4238)
X_val의 shape:  (60000, 4238)
 
val set's AUC score : 0.8012


In [11]:
# nom_8과 9 제외
train_sprs, test_sprs = cat_preprocessing(all_data, 8)
logistic_train(train_sprs, train_target)

nom_* 변수는 nom_0 부터 nom_7 까지 사용되었습니다
 
train data의 shape:  (300000, 2019)
test data의 shape:  (200000, 2019)
X_train의 shape:  (240000, 2019)
X_val의 shape:  (60000, 2019)
 
val set's AUC score : 0.7951


In [12]:
# nom_7, 8, 9 제외
train_sprs, test_sprs = cat_preprocessing(all_data, 7)
logistic_train(train_sprs, train_target)

nom_* 변수는 nom_0 부터 nom_6 까지 사용되었습니다
 
train data의 shape:  (300000, 799)
test data의 shape:  (200000, 799)
X_train의 shape:  (240000, 799)
X_val의 shape:  (60000, 799)
 
val set's AUC score : 0.7887


#### `nom_9`만 빼는 것이 가장 성능이 좋음을 알 수 있다. test_size 변수도 바꿔보자! 

In [13]:
# nom_9 제외, test_size 좀 더 줄여보기 
train_sprs, test_sprs = cat_preprocessing(all_data, 9)
logistic_train(train_sprs, train_target, tsize = 0.15)

nom_* 변수는 nom_0 부터 nom_8 까지 사용되었습니다
 
train data의 shape:  (300000, 4238)
test data의 shape:  (200000, 4238)
X_train의 shape:  (255000, 4238)
X_val의 shape:  (45000, 4238)
 
val set's AUC score : 0.8030


In [14]:
# nom_9 제외, test_size 좀 더 줄여보기 
train_sprs, test_sprs = cat_preprocessing(all_data, 9)
logistic_train(train_sprs, train_target, tsize = 0.1)

nom_* 변수는 nom_0 부터 nom_8 까지 사용되었습니다
 
train data의 shape:  (300000, 4238)
test data의 shape:  (200000, 4238)
X_train의 shape:  (270000, 4238)
X_val의 shape:  (30000, 4238)
 
val set's AUC score : 0.8051


학습셋(train set)에 사용되는 데이터를 더 많이 늘림으로써 AUC score도 높아지고 있다!  
최종 모델은 train / val set으로 나눈 게 아닌, 전체 `train_sprs` 데이터를 사용해 학습하고 `test_sprs`에 대해 예측해 제출해보자   

---
#### 이번엔 `n_iter` 값도 바꿔보자! (코드 실행 시간이 좀 더 걸리니 주의)

In [15]:
# nom_9 제외, test_size = 0.1, n_iter 값 늘리기
train_sprs, test_sprs = cat_preprocessing(all_data, 9)
logistic_train(train_sprs, train_target, tsize = 0.1, n_iter = 200)

nom_* 변수는 nom_0 부터 nom_8 까지 사용되었습니다
 
train data의 shape:  (300000, 4238)
test data의 shape:  (200000, 4238)
X_train의 shape:  (270000, 4238)
X_val의 shape:  (30000, 4238)
 
val set's AUC score : 0.8048


In [16]:
train_sprs, test_sprs = cat_preprocessing(all_data, 9)
logistic_train(train_sprs, train_target, tsize = 0.1, n_iter = 300)

nom_* 변수는 nom_0 부터 nom_8 까지 사용되었습니다
 
train data의 shape:  (300000, 4238)
test data의 shape:  (200000, 4238)
X_train의 shape:  (270000, 4238)
X_val의 shape:  (30000, 4238)
 
val set's AUC score : 0.8048


오잉? 예상과 달리 n_iter를 늘리니 성능이 더 안좋아졌다. 과적합의 가능성이 있으니, max_iter는 100을 유지하는 걸로!

## 제출 결과

In [17]:
# 전체 데이터로 학습
model = LogisticRegression(random_state = 99, max_iter = 100)
model.fit(train_sprs, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=99, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
# test data에 대해 예측 
y_pred = model.predict_proba(test_sprs)
y_pred[:, 1]

array([0.40607309, 0.67927933, 0.12011481, ..., 0.32968039, 0.67057358,
       0.22289988])

In [19]:
sub = pd.read_csv('sample_submission.csv')
del sub['target']
sub['target'] = y_pred[:, 1]
sub

,id,target
0,300000,0.406073
1,300001,0.679279
2,300002,0.120115
3,300003,0.511687
4,300004,0.873930
...,...,...
199995,499995,0.264724
199996,499996,0.137941
199997,499997,0.329680
199998,499998,0.670574


In [20]:
sub.to_csv('sub_sy_nom.csv', index = False)

### test data에 대한 Score
- **public: 0.80421 / private: 0.79823** 
- 베이스라인 모델 0.802보다 → 0.804로 꽤 성능 향상을 보였다!

--> 결과를 반영하여, 앞으로는 nom_9 변수만 제거해 모델링에 사용하도록 하자!